In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark

import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf


import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark


import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.SparkContext

import org.apache.spark.SparkConf


In [2]:
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Column

import org.apache.spark.sql.types.{DataType, DateType, TimestampType}

import org.apache.spark.sql.functions._

In [3]:
var spark = SparkSession
.builder()
.appName("Java Spark SQL basic example")
.config("spark.master", "local")
.getOrCreate();

def sc = spark.sparkContext


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@2a7697d3
defined function sc

In [4]:
val df = spark.read.option("header", true).option("delimiter", ",").option("inferSchema",true).csv("homeworkfile.csv")

df: DataFrame = [country_code: string, device_id: string ... 12 more fields]

In [5]:
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}


import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@78d3b14f
import scala.util.Try

import java.sql.{Connection, DriverManager, ResultSet}


In [6]:
/* 2. which tool has highest return rate (edit_text_apply, effect_apply, 
edit_item_open, edit_sticker_apply)
*/

import org.apache.spark.sql._

// In df1 a new "date" column is added and corresponding "event_type" is filtered in df (our basic DB of)
var df1 =  df.withColumn("date",to_date(col("timestamp"))).where( col("event_type") === "edit_text_apply" ||
                      col("event_type") === "effect_apply" ||
                      col("event_type") === "edit_item_open" ||
                      col("event_type") === "edit_sticker_apply")


// In filtereddf1 we have grouped the df1 by date and pivoted event_types,
// filtered the result and selected only  the first and last day of the period (where statement)

var filtereddf1 = df1.groupBy("date").pivot("event_type").agg(countDistinct(col("device_id")))
.sort("date").where(col("date") === max(col("date")) || col("date") === min(col("date")))

// I have stuck here and cannot make operations for each row 



import org.apache.spark.sql._

// In df1 a new "date" column is added and corresponding "event_type" is filtered in df (our basic DB of)

df1: Dataset[Row] = [country_code: string, device_id: string ... 13 more fields]
filtereddf1: Dataset[Row] = [date: date, edit_item_open: bigint ... 3 more fields]

In [12]:
/*3. how much users did photo_like daily, in which day is it highest, if there is seasonality*/

// Add date column to show date of event, filter df and left only rows with event “photo_like”
var filteredDF = df.withColumn("Date",to_date(col("timestamp"))).filter(col("event_type") === "photo_like")

//group by date filteredDF and add column “Day of week”  and check weekly seasonality
var groupedDF = filteredDF.groupBy("Date").agg(count("event_type").alias("Number of photo_likes")).withColumn("Day_of_week", date_format(col("Date"), "u")).sort("Date")

groupedDF.groupBy("Day_of_week").agg(avg("Number of photo_likes")).sort("Day_of_week").show()


+-----------+--------------------------+
|Day_of_week|avg(Number of photo_likes)|
+-----------+--------------------------+
|          1|                       2.5|
|          2|                       4.5|
|          3|                      3.25|
|          4|                      1.75|
|          5|                       3.4|
|          6|                      3.25|
|          7|                       3.0|
+-----------+--------------------------+



filteredDF: Dataset[Row] = [country_code: string, device_id: string ... 13 more fields]
groupedDF: Dataset[Row] = [Date: date, Number of photo_likes: bigint ... 1 more field]

In [8]:
/*4. what is conversion rate that users who did effect_try then did effect apply*/
// Count the number of “effect_try” in event_type column – eff_try

var eff_try = count(when(col("event_type") === "effect_try",true))

// Count the number of “effect_apply” in event_type column – eff_apply
var eff_apply = count(when(col("event_type") === "effect_apply",true))

//Calculate conversion ratio by initially dropping all the null values from “user_id” column and show results
df.na.drop("any",Seq("user_id")).select(eff_try as "Number of effect_try",eff_apply as "Number of effect_apply" ,eff_apply/eff_try*100 as "Conversion rate(%)").show()




+--------------------+----------------------+------------------+
|Number of effect_try|Number of effect_apply|Conversion rate(%)|
+--------------------+----------------------+------------------+
|               16141|                  2019| 12.50851867914008|
+--------------------+----------------------+------------------+



eff_try: Column = count(CASE WHEN (event_type = effect_try) THEN true END)
eff_apply: Column = count(CASE WHEN (event_type = effect_apply) THEN true END)

In [13]:
/*5. find how many users did effect apply and what is descriptive statistics for it (outliers, avg, median)*/

// Add 2 columns "Date" and "Day_of_week" to our dataFrame - newDF
var newDF = df.withColumn("Date",to_date(col("timestamp"))).withColumn("Day_of_week",date_format(col("Date"),"u"))

// Create group newDF by  "Date" and "Day_of_week" columns aggregating with "event_type"
// where "event_type" is "effect_apply" and dropping all Null values genereted by cube method
var groupedDF = newDF.cube("Date","Day_of_week").agg(count(when(col("event_type") === "effect_apply",true)) as "Number of effect_applies").na.drop().sort("Date")

// Generate general statistic  by describe method
groupedDF.describe("Number of effect_applies").show()

// Assuming that outliers here are "mean - 2*stddev" < row < "mean + 2*stddev" 
groupedDF.where(col("Number of effect_applies") > 
                (mean(col("Number of effect_applies")) + 2*(stddev(col("Number of effect_applies"))))
                 || col("Number of effect_applies") < (mean(col("Number of effect_applies")
                    - 2*(stddev(col("Number of effect_applies")))))).show()

//We have one outlier on 2018-02-11 (sunday) with 189 used effects

+-------+------------------------+
|summary|Number of effect_applies|
+-------+------------------------+
|  count|                      29|
|   mean|      128.06896551724137|
| stddev|      19.033675109587744|
|    min|                      97|
|    max|                     179|
+-------+------------------------+

+----------+-----------+------------------------+
|      Date|Day_of_week|Number of effect_applies|
+----------+-----------+------------------------+
|2018-02-08|          4|                     100|
|2018-02-11|          7|                     179|
|2018-02-23|          5|                      97|
+----------+-----------+------------------------+



newDF: DataFrame = [country_code: string, device_id: string ... 14 more fields]
groupedDF: Dataset[Row] = [Date: date, Day_of_week: string ... 1 more field]

In [13]:
/* 7 What are the most important metrics when we want to describe one session of user in app 

Depends on what we mean by saying important, for data scienteis more important metrics to my opinion are:

1. Duration of the session 
2. What events were created (i.e. tools used) by user during session
3. Sequence of events during a session
4. The opportunity to place the session in any cluster (hope you have any :), otherwise think of it)
5. How many events were completed



*/